In [2]:
import pandas as pd

orders = pd.read_csv("../data/orders.csv")
users = pd.read_json("../data/users.json")

orders.head(), users.head()

(   order_id  user_id  restaurant_id  order_date  total_amount  \
 0         1     2508            450  18-02-2023        842.97   
 1         2     2693            309  18-01-2023        546.68   
 2         3     2084            107  15-07-2023        163.93   
 3         4      319            224  04-10-2023       1155.97   
 4         5     1064            293  25-12-2023       1321.91   
 
                   restaurant_name  
 0               New Foods Chinese  
 1  Ruchi Curry House Multicuisine  
 2           Spice Kitchen Punjabi  
 3          Darbar Kitchen Non-Veg  
 4       Royal Eatery South Indian  ,
    user_id    name       city membership
 0        1  User_1    Chennai    Regular
 1        2  User_2       Pune       Gold
 2        3  User_3  Bangalore       Gold
 3        4  User_4  Bangalore    Regular
 4        5  User_5       Pune       Gold)

In [3]:
import sqlite3

conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

with open("../data/restaurants.sql", "r", encoding="utf-8") as f:
    cursor.executescript(f.read())

restaurants = pd.read_sql("SELECT * FROM restaurants", conn)

restaurants.head()

,restaurant_id,restaurant_name,cuisine,rating
0,1,Restaurant_1,Chinese,4.8
1,2,Restaurant_2,Indian,4.1
2,3,Restaurant_3,Mexican,4.3
3,4,Restaurant_4,Chinese,4.1
4,5,Restaurant_5,Chinese,4.8


In [4]:
restaurants.info()

<class 'pandas.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   restaurant_id    500 non-null    int64  
 1   restaurant_name  500 non-null    str    
 2   cuisine          500 non-null    str    
 3   rating           500 non-null    float64
dtypes: float64(1), int64(1), str(2)
memory usage: 15.8 KB


In [5]:
orders['order_date'] = pd.to_datetime(orders['order_date'], format="%d-%m-%Y")
orders.info()

<class 'pandas.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   order_id         10000 non-null  int64         
 1   user_id          10000 non-null  int64         
 2   restaurant_id    10000 non-null  int64         
 3   order_date       10000 non-null  datetime64[us]
 4   total_amount     10000 non-null  float64       
 5   restaurant_name  10000 non-null  str           
dtypes: datetime64[us](1), float64(1), int64(3), str(1)
memory usage: 468.9 KB


In [6]:
orders.rename(columns={'total_amount': 'amount'}, inplace=True)
users.rename(columns={'membership': 'membership_type'}, inplace=True)

In [7]:
final_df = (
    orders
    .merge(users, on="user_id", how="left")
    .merge(restaurants, on="restaurant_id", how="left")
)

final_df.head()

,order_id,user_id,restaurant_id,order_date,amount,restaurant_name_x,name,city,membership_type,restaurant_name_y,cuisine,rating
0,1,2508,450,2023-02-18,842.97,New Foods Chinese,User_2508,Hyderabad,Regular,Restaurant_450,Mexican,3.2
1,2,2693,309,2023-01-18,546.68,Ruchi Curry House Multicuisine,User_2693,Pune,Regular,Restaurant_309,Indian,4.5
2,3,2084,107,2023-07-15,163.93,Spice Kitchen Punjabi,User_2084,Chennai,Gold,Restaurant_107,Mexican,4.0
3,4,319,224,2023-10-04,1155.97,Darbar Kitchen Non-Veg,User_319,Bangalore,Gold,Restaurant_224,Chinese,4.8
4,5,1064,293,2023-12-25,1321.91,Royal Eatery South Indian,User_1064,Pune,Regular,Restaurant_293,Italian,3.0


In [8]:
final_df.shape

(10000, 12)

In [9]:
final_df.isnull().sum()

order_id             0
user_id              0
restaurant_id        0
order_date           0
amount               0
restaurant_name_x    0
name                 0
city                 0
membership_type      0
restaurant_name_y    0
cuisine              0
rating               0
dtype: int64

In [10]:
final_df.to_csv("../output/final_food_delivery_dataset.csv", index=False)

In [11]:
final_df.shape

(10000, 12)

In [12]:
orders_by_month = (
    final_df
    .groupby(final_df['order_date'].dt.to_period("M"))
    .size()
)

orders_by_month

order_date
2023-01    804
2023-02    785
2023-03    903
2023-04    812
2023-05    844
2023-06    784
2023-07    859
2023-08    851
2023-09    812
2023-10    863
2023-11    807
2023-12    849
2024-01     27
Freq: M, dtype: int64

In [15]:
orders_by_month

order_date
2023-01    804
2023-02    785
2023-03    903
2023-04    812
2023-05    844
2023-06    784
2023-07    859
2023-08    851
2023-09    812
2023-10    863
2023-11    807
2023-12    849
2024-01     27
Freq: M, dtype: int64

In [16]:
final_df.groupby("membership_type")["amount"].agg(["count", "mean", "sum"])

,count,mean,sum
membership_type,,,
Gold,4987,797.145556,3975364.89
Regular,5013,805.158434,4036259.23


In [17]:
final_df.groupby("city")["amount"].sum().sort_values(ascending=False)


city
Bangalore    2206946.58
Chennai      1990513.03
Pune         1924797.93
Hyderabad    1889366.58
Name: amount, dtype: float64

In [32]:
final_df.groupby("cuisine")["amount"].mean().sort_values(ascending=False)

cuisine
Mexican    808.021344
Italian    799.448578
Indian     798.466011
Chinese    798.389020
Name: amount, dtype: float64

In [21]:
final_df.groupby("restaurant_name_y")["amount"].sum().sort_values(ascending=False).head(10)

restaurant_name_y
Restaurant_287    29460.47
Restaurant_19     29289.52
Restaurant_484    27051.49
Restaurant_61     26049.71
Restaurant_134    25791.23
Restaurant_440    25467.45
Restaurant_383    24955.59
Restaurant_188    24698.82
Restaurant_431    24684.31
Restaurant_153    24523.66
Name: amount, dtype: float64

In [20]:
final_df.columns

Index(['order_id', 'user_id', 'restaurant_id', 'order_date', 'amount',
       'restaurant_name_x', 'name', 'city', 'membership_type',
       'restaurant_name_y', 'cuisine', 'rating'],
      dtype='str')

In [22]:
user_spend = final_df.groupby("user_id")["amount"].sum()
(user_spend > 1000).sum()

np.int64(2544)

In [23]:
final_df.groupby(pd.cut(final_df["rating"],
        bins=[3.0,3.5,4.0,4.5,5.0]))["amount"].sum()

rating
(3.0, 3.5]    1881754.57
(3.5, 4.0]    1717494.41
(4.0, 4.5]    1960326.26
(4.5, 5.0]    2197030.75
Name: amount, dtype: float64

In [24]:
final_df[final_df["membership_type"]=="Gold"] \
    .groupby("city")["amount"].mean().sort_values(ascending=False)

city
Chennai      808.459080
Hyderabad    806.421034
Bangalore    793.223756
Pune         781.162243
Name: amount, dtype: float64

In [25]:
final_df.groupby("cuisine")["restaurant_id"].nunique()

cuisine
Chinese    120
Indian     126
Italian    126
Mexican    128
Name: restaurant_id, dtype: int64

In [26]:
(final_df["membership_type"]=="Gold").mean()*100

np.float64(49.87)

In [29]:
restaurant_stats = final_df.groupby("restaurant_name_x").agg(
    avg_amount=("amount","mean"),
    order_count=("order_id","count")
)

restaurant_stats[restaurant_stats["order_count"]<20] \
    .sort_values("avg_amount",ascending=False).head()

,avg_amount,order_count
restaurant_name_x,,
Hotel Dhaba Multicuisine,1040.222308,13
Sri Mess Punjabi,1029.180833,12
Ruchi Biryani Punjabi,1002.140625,16
Sri Delights Pure Veg,989.467222,18
Classic Kitchen Family Restaurant,973.167895,19


In [30]:
final_df.groupby(["membership_type","cuisine"])["amount"].sum() \
    .sort_values(ascending=False)

membership_type  cuisine
Regular          Mexican    1072943.30
                 Italian    1018424.75
Gold             Mexican    1012559.79
                 Italian    1005779.05
Regular          Indian      992100.27
Gold             Indian      979312.31
                 Chinese     977713.74
Regular          Chinese     952790.91
Name: amount, dtype: float64

In [31]:
final_df.groupby(final_df["order_date"].dt.quarter)["amount"].sum()

order_date
1    2010626.64
2    1945348.72
3    2037385.10
4    2018263.66
Name: amount, dtype: float64

In [33]:
final_df["user_id"].nunique()

2883

In [34]:
final_df[final_df["rating"] >= 4.5].shape[0]

3374

In [35]:
final_df[
    (final_df["membership_type"]=="Gold") &
    (final_df["city"]=="Bangalore")
].shape[0]

1254